In [9]:
import sqlite3
from typing import Dict, List, Tuple, Any
from dataclasses import dataclass
import json
import os 
from pathlib import Path

In [10]:
# Databas connection 
dp_abs_path = Path(r"data\northwind.sqlite")
dp_path = os.path.join(os.getcwd(),dp_abs_path)
conn = sqlite3.connect(dp_path)
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

In [ ]:
# Create views 
try:
    cursor.execute('CREATE VIEW IF NOT EXISTS order_items AS SELECT * FROM "Order Details";')
    conn.commit()

except Exception as e:
    print("Error creating view : {e}")

In [47]:
# Database Introspection

# Get all needed table names
cursor.execute("""
SELECT name FROM sqlite_master
WHERE type IN ('table','view') AND name NOT LIKE 'sqlite_%'
ORDER BY name
""")
valid_views = ['categories','order_items','orders','products','customers']

tables = [row[0] for row in cursor.fetchall() if row[0].lower() in valid_views]
print("Avaliable table names : ")
for table in tables:
    print(table)

schema = {}

for table in tables: 
    cursor.execute(f"PRAGMA table_info(\"{table}\")")
    columns = [(row[1],row[2]) for row in cursor.fetchall()]
    schema[table]=columns

for key,value in schema.items():
    print(f"({key} : {value})")


Avaliable table names : 
Categories
Customers
Orders
Products
order_items
(Categories : [('CategoryID', 'INTEGER'), ('CategoryName', 'TEXT'), ('Description', 'TEXT'), ('Picture', 'BLOB')])
(Customers : [('CustomerID', 'TEXT'), ('CompanyName', 'TEXT'), ('ContactName', 'TEXT'), ('ContactTitle', 'TEXT'), ('Address', 'TEXT'), ('City', 'TEXT'), ('Region', 'TEXT'), ('PostalCode', 'TEXT'), ('Country', 'TEXT'), ('Phone', 'TEXT'), ('Fax', 'TEXT')])
(Orders : [('OrderID', 'INTEGER'), ('CustomerID', 'TEXT'), ('EmployeeID', 'INTEGER'), ('OrderDate', 'DATETIME'), ('RequiredDate', 'DATETIME'), ('ShippedDate', 'DATETIME'), ('ShipVia', 'INTEGER'), ('Freight', 'NUMERIC'), ('ShipName', 'TEXT'), ('ShipAddress', 'TEXT'), ('ShipCity', 'TEXT'), ('ShipRegion', 'TEXT'), ('ShipPostalCode', 'TEXT'), ('ShipCountry', 'TEXT')])
(Products : [('ProductID', 'INTEGER'), ('ProductName', 'TEXT'), ('SupplierID', 'INTEGER'), ('CategoryID', 'INTEGER'), ('QuantityPerUnit', 'TEXT'), ('UnitPrice', 'NUMERIC'), ('UnitsInStock',

In [57]:
 # approximate tokens length of the db schema 
 x = json.dumps(schema)
 print("Approximate number of tokens : ")
 print(len(x)/3)

Approximate number of tokens : 
388.0


In [61]:
""" Remove unnecessary entities linked with :
eployee table , shipment table , supplier table for simplicity """

unwanted = ['supplierid','shipperid','picture','employeeid',
'freight','shipname','shippostalcode']

new_schema = {}
for key,value in schema.items():
    temp_list = []
    for item in value:
        if item[0].lower() in unwanted:
            continue
        temp_list.append((item[0],item[1]))

    new_schema[key] = temp_list

x = json.dumps(new_schema)
print("Approximate number of tokens :")
print(len(x)/3)

Approximate number of tokens :
338.3333333333333


In [11]:
# CHECK HOW DATES ARE STORED 

# Print first 10 OrderDate values
cursor.execute("""
    SELECT OrderID, OrderDate
    FROM Orders
    LIMIT 10;
""")

rows = cursor.fetchall()

print("Sample OrderDate values:")
for row in rows:
    print(dict(row))



Sample OrderDate values:
{'OrderID': 10248, 'OrderDate': '2016-07-04'}
{'OrderID': 10249, 'OrderDate': '2016-07-05'}
{'OrderID': 10250, 'OrderDate': '2016-07-08'}
{'OrderID': 10251, 'OrderDate': '2016-07-08'}
{'OrderID': 10252, 'OrderDate': '2016-07-09'}
{'OrderID': 10253, 'OrderDate': '2016-07-10'}
{'OrderID': 10254, 'OrderDate': '2016-07-11'}
{'OrderID': 10255, 'OrderDate': '2016-07-12'}
{'OrderID': 10256, 'OrderDate': '2016-07-15'}
{'OrderID': 10257, 'OrderDate': '2016-07-16'}


In [13]:
cursor.execute("""
    SELECT OrderID, OrderDate
    FROM Orders
    WHERE strftime('%Y', OrderDate) = '2016'
    LIMIT 5;
""")
rows = cursor.fetchall()


for row in rows:
    print(dict(row))



{'OrderID': 10248, 'OrderDate': '2016-07-04'}
{'OrderID': 10249, 'OrderDate': '2016-07-05'}
{'OrderID': 10250, 'OrderDate': '2016-07-08'}
{'OrderID': 10251, 'OrderDate': '2016-07-08'}
{'OrderID': 10252, 'OrderDate': '2016-07-09'}


In [67]:
with open('data/db_schema.json','w') as f:
    json.dump(new_schema,f)